* 사업보고서의 "재무제표 주석" 문서의 소제목 리스트 뽑기

In [4]:
def cleanPage(page):
    # 줄바꿈없는 공백(Non-breaking Space) 표식 제거
    page = page.replace('&nbsp;', ' ')
    # span tag 제거
    page = page.replace('<span>', '')
    page = page.replace('</span>', '')
    page = page.replace('<SPAN>', '')
    page = page.replace('</SPAN>', '')
    return page

In [209]:
from bs4 import BeautifulSoup
import re

file_path = "D:/PythonProject/data-gatherer/dart_fs_notes/sample/큐젠바이오텍_2020(특이Case).html"
with open(file_path,'rt', encoding='utf-8') as file:
    page = file.read() 

page = cleanPage(page)
soup = BeautifulSoup(page)
tags = soup.find_all(string = True)
subtitle_list = []
prev_number = 0
for t in tags:
    subjects = re.findall('\d+[.]+\s+[^0-9]+', t.text)  # 숫자+'.'+공백+숫자가아닌모든문자열 
    # subjects = re.findall('\d+[.]+\s+[가-힣]+', t.text)  # 숫자+'.'+공백+문자열(한글)  
    # subjects = re.findall('\d+[.]+\s+\w+', t.text)  # 숫자+'.'+공백+문자열(한글)      
    if len(subjects) > 0:
        for s in subjects:
            if not s.__contains__('주석'):
                this_number = int(s.split(".")[0])
                if this_number > prev_number:
                    subtitle_list.append(s)
                    prev_number = this_number

In [210]:
subtitle_list

['1. 회사의 개요',
 '2. 중요한 회계처리 방침',
 '3.  사용이 제한된 금융상품',
 '4. 매도가능증권',
 '5. 유형자산 ',
 '6. 무형자산 ',
 '7. 장기차입금 ',
 '8. 신주인수권부사채',
 '9. 우발부채와 약정사항 등 ',
 '10. 퇴직급여충당부채 ',
 '11. 자본금 ',
 '12. 주식매수선택권',
 '13. 결손금처리계산서 ',
 '14.  법인세등',
 '15. 특수관계자와의 거래',
 '16. 금융리스 ',
 '17. 외화자산ㆍ부채 ',
 '18. 부가가치계산에 필요한 계정',
 '19. 현금흐름표',
 '20. 유상증자',
 '21. 재무제표의 승인 ']

In [5]:
from bs4 import BeautifulSoup
import re
import pandas as pd

file_path = "D:/PythonProject/data-gatherer/dart_fs_notes/sample/HSD엔진_2022.html"
with open(file_path,'rt', encoding='utf-8') as file:
    page = file.read()      # html document
    
page = cleanPage(page)
soup = BeautifulSoup(page)
# tags = soup.find_all('p')
tags = soup.find_all(string = True)
sub_title_list = []
for t in tags:
    subjects = re.findall('\d+[.]+\s+[^0-9]+', t.text)  # 숫자+'.'+공백+숫자가아닌모든문자열      
    if len(subjects) > 0:
        for s in subjects:
            if not s.__contains__('주석'):
                sub_title_list.append(s)
            # keywords: 퇴직급여채무, 순확정급여부채, 퇴직급여충당부채
            if s.__contains__('순확정급여부채'):
                this_title = s
                start_no = this_title.split(".")[0]
                end_no = str(int(start_no) + 1)
                print(start_no, end_no)

next_title = None
for st in sub_title_list:
    if st.startswith(end_no):
        next_title = st
        
print(this_title)
print(next_title)

h = soup.prettify()
p0 = h.find(this_title)
p1 = h.find(next_title)

# p0 = page.find(this_title)
# p1 = page.find(next_title)
print(p0, p1)

part_html = h[p0:p1]     # 문서 Slicing

20 21
20. 순확정급여부채
21. 충당부채
260378 280757


In [6]:
part_html

'20. 순확정급여부채\n   <br/>\n   <br/>\n   당사는 확정급여형 퇴직급여제도를 운영하고 있으며, 사외적립자산과 확정급여채무의 보험수리적 평가는 예측단위적립방식을 사용하여 적격성이 있는 독립적인 보험계리법인에 의하여 수행되었습니다.\n   <br/>\n   <br/>\n   (1) 당기말과 전기말 현재 순확정급여부채는 다음과 같습니다.\n  </p>\n  <table class="nb" width="603">\n   <colgroup>\n    <col width="603"/>\n   </colgroup>\n   <tbody>\n    <tr>\n     <td align="RIGHT" height="20" valign="TOP" width="603">\n      (단위 : 천원)\n     </td>\n    </tr>\n   </tbody>\n  </table>\n  <table border="1" width="604">\n   <colgroup>\n    <col width="202"/>\n    <col width="201"/>\n    <col width="201"/>\n   </colgroup>\n   <thead>\n    <tr>\n     <th align="CENTER" height="20" width="202">\n      구분\n     </th>\n     <th align="CENTER" height="20" width="201">\n      당기말\n     </th>\n     <th align="CENTER" height="20" width="201">\n      전기말\n     </th>\n    </tr>\n   </thead>\n   <tbody>\n    <tr>\n     <td height="20" valign="TOP" width="202">\n      확정급여채무의 현재가치\n     </td>\n     <td align="RIGHT" height="20" valign="TOP" wid

In [7]:
data_list = pd.read_html(part_html)

In [8]:
data_list

[           0
 0  (단위 : 천원),
              구분           당기말           전기말
 0  확정급여채무의 현재가치      35390325      38074012
 1  사외적립자산의 공정가치  (30,572,150)  (28,803,767)
 2       순확정급여부채       4818175       9270245,
            0
 0  (단위 : 천원),
                    구분       당기       전기
 0              당기근무원가  3609386  3589851
 1  순이자비용(이자비용 - 기대수익)   250061   136428
 2                  합계  3859447  3726279,
            0
 0  (단위 : 천원),
                 구분       당기       전기
 0             매출원가  2787337  2720399
 1          판매비와관리비   913212   872521
 2  판매비와관리비-경상연구개발비   158898   133359
 3               합계  3859447  3726279,
            0
 0  (단위 : 천원),
               구분           당기           전기
 0             기초     38074012     35706854
 1         당기근무원가      3609386      3589851
 2           이자비용      1202695       943751
 3  확정급여채무의 재측정요소  (4,745,946)      1436575
 4  - 인구통계적가정의 변동       511240       982777
 5    - 재무적가정의 변동  (4,862,066)  (1,718,494)
 6       - 경험조정 등    (395,120)      217